In [36]:
from webscrape_fast import *
import json
from concurrent.futures import ThreadPoolExecutor
from tqdm.auto import tqdm

In [2]:
countries_urls = get_countries_urls()

In [75]:
with open('data/countries_urls.json', 'w') as f:
    json.dump(countries_urls, f, indent=4)

In [3]:
executor = ThreadPoolExecutor()
futures = [executor.submit(get_regions_urls, url) for url in countries_urls]
regions_urls = []
for f in tqdm(futures):
    regions_urls += f.result()

100%|██████████| 246/246 [00:34<00:00,  7.16it/s]


In [73]:
with open('data/regions_urls.json', 'w') as f:
    json.dump(regions_urls, f, indent=4)

In [26]:
def get_all_cities_urls(regions_cities_urls=[]):
    executor = ThreadPoolExecutor()
    if not regions_cities_urls:
        futures = [executor.submit(get_cities_urls, url) for url in regions_urls]
        for f in tqdm(futures):
            try:
                regions_cities_urls.append(f.result())
            except:
                regions_cities_urls.append([])
                print("failed to get cites urls from region nr", i)
    else:
        futures = [(i, executor.submit(get_cities_urls, url)) for i, url in filter(lambda x: not x[1], enumerate(regions_urls))]
        for f in tqdm(futures):
            try:
                regions_cities_urls[f[0]] = f[1].result()
            except:
                print("failed to get cites urls from region nr", f[0])
    return regions_cities_urls

In [62]:
def get_all_cities_urls(regions_cities_urls=[]): # better solution
    if not regions_cities_urls:
        for _ in regions_urls:
            regions_cities_urls.append([])

    executor = ThreadPoolExecutor()
    futures = [(i, executor.submit(get_cities_urls, regions_urls[i])) for i, _ in filter(lambda x: x[1] is None, enumerate(regions_cities_urls))]
    for i, f in tqdm(futures):
        try:
            regions_cities_urls[i] = f.result()
        except requests.exceptions.RequestException as e:
            regions_cities_urls[i] = None
            print("Failed! : request exception, url nr", i)
        except:
            regions_cities_urls[i] = None
            print("Failed! : something else, url nr", i)
    return regions_cities_urls

In [27]:
regions_cities_urls = []

In [63]:
regions_cities_urls = get_all_cities_urls(regions_cities_urls)

0it [00:00, ?it/s]


In [60]:
with open('data/regions_cities_urls.json', 'w') as f:
    json.dump(regions_cities_urls, f, indent=4)

In [61]:
sum = 0
for cities in regions_cities_urls:
    sum += len(cities)
print("sum =", sum)

sum = 304279


In [78]:
not_loaded = []
for i, _ in filter(lambda x: not x[1], enumerate(regions_cities_urls)):
    not_loaded.append("https://tcktcktck.org" + regions_urls[i] + "/locations")

In [79]:
print(len(not_loaded))
print(not_loaded)

93
['https://tcktcktck.org/andorra/escaldes-engordany/locations', 'https://tcktcktck.org/antigua-and-barbuda/redonda/locations', 'https://tcktcktck.org/azerbaijan/shirvan/locations', 'https://tcktcktck.org/bermuda/saint-george/locations', 'https://tcktcktck.org/bermuda/southampton/locations', 'https://tcktcktck.org/botswana/francistown/locations', 'https://tcktcktck.org/botswana/jwaneng/locations', 'https://tcktcktck.org/botswana/lobatse/locations', 'https://tcktcktck.org/france/clipperton-island/locations', 'https://tcktcktck.org/french-southern-territories/bassas-da-india/locations', 'https://tcktcktck.org/french-southern-territories/europa-island/locations', 'https://tcktcktck.org/french-southern-territories/juan-de-nova-island/locations', 'https://tcktcktck.org/french-southern-territories/saint-paul-island/locations', 'https://tcktcktck.org/french-southern-territories/tromelin-island/locations', 'https://tcktcktck.org/guam/umatac/locations', 'https://tcktcktck.org/guernsey/herm/loc

In [76]:
cities_urls = []
for cities in regions_cities_urls:
    cities_urls += cities

In [77]:
with open('data/cities_urls.json', 'w') as f:
    json.dump(cities_urls, f, indent=4)